## Imports

In [1]:
import albumentations as A

import numpy as np
import torch


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#DEVICE = 'cpu'
BATCH_SIZE = 1
import torch

torch.set_float32_matmul_precision("high")
#import armory.data.datasets
import datasets


In [27]:
LABEL_MAP = {
    11: 0,  # Fixed-wing Aircraft
    12: 1,  # Small Aircraft
    13: 2,  # Cargo Plane
    15: 3,  # Helicopter
    17: 4,  # Passenger Vehicle
    18: 5,  # Small Car
    19: 6,  # Bus
    20: 7,  # Pickup Truck
    21: 8,  # Utility Truck
    23: 9,  # Truck
    24: 10,  # Cargo Truck
    25: 11,  # Truck w/Box
    26: 12,  # Truck Tractor
    27: 13,  # Trailer
    28: 14,  # Truck w/Flatbed
    29: 15,  # Truck w/Liquid
    32: 16,  # Crane Truck
    33: 17,  # Railway Vehicle
    34: 18,  # Passenger Car
    35: 19,  # Cargo Car
    36: 20,  # Flat Car
    37: 21,  # Tank Car
    38: 22,  # Locomotive
    40: 23,  # Maritime Vessel
    41: 24,  # Motorboat
    42: 25,  # Sailboat
    44: 26,  # Tugboat
    45: 27,  # Barge
    47: 28,  # Fishing Vessel
    49: 29,  # Ferry
    50: 30,  # Yacht
    51: 31,  # Container Ship
    52: 32,  # Oil Tanker
    53: 33,  # Engineering Vehicle
    54: 34,  # Tower Crane
    55: 35,  # Container Crane
    56: 36,  # Reach Stacker
    57: 37,  # Straddle Carrier
    59: 38,  # Mobile Crane
    60: 39,  # Dump Truck
    61: 40,  # Haul Truck
    62: 41,  # Scraper/Tractor
    63: 42,  # Front Loader/Bulldozer
    64: 43,  # Excavator
    65: 44,  # Cement Mixer
    66: 45,  # Ground Grader
    71: 46,  # Hut/Tent
    72: 47,  # Shed
    73: 48,  # Building
    74: 49,  # Aircraft Hanger
    75: 50,  # Unknown1
    76: 51,  # Damaged Building
    77: 52,  # Facility
    79: 53,  # Construction Site
    82: 54,  # Unknown2
    83: 55,  # Vehicle Lot
    84: 56,  # Helipad
    86: 57,  # Storage Tank
    89: 58,  # Shipping Container Lot
    91: 59,  # Shipping Container
    93: 60,  # Pylon
    94: 61,  # Tower
}

## Load Data

In [28]:
def load_huggingface_dataset():
    train_data = datasets.load_dataset("Honaker/xview_dataset", split="train")
    new_dataset = train_data.train_test_split(test_size=0.3, seed=1)
    #new_dataset = train_data['train'].train_test_split(test_size=0.5, seed=1)
    train_dataset, test_dataset = new_dataset["train"], new_dataset["test"]

    #train_dataset, test_dataset = HuggingFaceObjectDetectionDataset(
    #    train_dataset
    #), HuggingFaceObjectDetectionDataset(test_dataset)

    return train_dataset, test_dataset
train_dataset, test_dataset = load_huggingface_dataset()


Found cached dataset parquet (/home/chris/.cache/huggingface/datasets/Honaker___parquet/Honaker--xview_dataset-4b2c80dc283ac8d5/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
Loading cached split indices for dataset at /home/chris/.cache/huggingface/datasets/Honaker___parquet/Honaker--xview_dataset-4b2c80dc283ac8d5/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-39dde171fdd76164.arrow and /home/chris/.cache/huggingface/datasets/Honaker___parquet/Honaker--xview_dataset-4b2c80dc283ac8d5/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-e21aaff49de2d824.arrow


In [29]:
from albumentations.pytorch import ToTensorV2

def get_train_transform():
    return A.Compose([
        #A.Flip(0.5),
        #A.RandomRotate90(0.5),
        #A.MotionBlur(p=0.2),
        #A.MedianBlur(blur_limit=3, p=0.1),
        #A.Blur(blur_limit=3, p=0.1),
        ToTensorV2(p=1.0),
    ], bbox_params={
        'format': 'pascal_voc',
        'label_fields': ['labels']
    })
# define the validation transforms
def get_valid_transform():
    return A.Compose([
        ToTensorV2(p=1.0),
    ], bbox_params={
        'format': 'pascal_voc', 
        'label_fields': ['labels']
    })


In [30]:
import os
import torch

from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
#from torchvision import tv_tensors
import PIL
from torchvision.transforms.v2 import functional as F
import cv2


class xview(torch.utils.data.Dataset):
    def __init__(self,dataset, width, height,  transforms=None):
        self.transforms = transforms
        self.image_dataset = dataset
        self.height = height
        self.width = width
  

    def __getitem__(self, idx):
        img = self.image_dataset[idx]


        image = np.asarray(img['image'])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image_resized = cv2.resize(image, (self.width, self.height))
        image_resized /= 255.0
        image_width = image.shape[1]
        image_height = image.shape[0]

        boxes = img['objects']['bbox']
        box_final = []
        for box in boxes:
            xmin_final = (box[0]/image_width)*self.width
            xmax_final = (box[2]/image_width)*self.width
            ymin_final = (box[1]/image_height)*self.height
            yamx_final = (box[3]/image_height)*self.height
            box_final.append([xmin_final, ymin_final, xmax_final, yamx_final])


        boxes = torch.as_tensor(box_final, dtype=torch.float32)
        area = img['objects']['area']
        area = torch.as_tensor(area, dtype=torch.float32)
        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)
        labels = img['objects']['category']
        labels = list(
                map(lambda x: LABEL_MAP[x], labels)
            )  # map original classes to sequential classes
        labels = torch.as_tensor(labels, dtype=torch.int64)

        target = {}
        target["bboxes"] = boxes
        target["labels"] = labels
        #target["area"] = area
        #target["iscrowd"] = iscrowd
        image_id = torch.tensor([idx])
        target["image_id"] = image_id
        target["img_size"] =  (self.height, self.width),
        target["img_scale"] = torch.tensor([1.0]),


        if self.transforms:
            sample = self.transforms(image = image_resized,
                                     bboxes = target['bboxes'],
                                     labels = labels)
            image_resized = sample['image']
            target['bboxes'] = torch.Tensor(sample['bboxes'])
            
        return image_resized, target, image_id



    def __len__(self):
        return len(self.image_dataset)

In [31]:

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2


def get_train_transforms(target_img_size=512):
    return A.Compose(
        [
            #A.HorizontalFlip(p=0.5),
            A.Resize(height=target_img_size, width=target_img_size, p=1),
            A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ToTensorV2(p=1),
        ],
        p=1.0,
        bbox_params=A.BboxParams(
            format="pascal_voc", min_area=0, min_visibility=0, label_fields=["labels"]
        ),
    )


def get_valid_transforms(target_img_size=512):
    return A.Compose(
        [
            #A.Resize(height=target_img_size, width=target_img_size, p=1),
            A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ToTensorV2(p=1),
        ],
        p=1.0,
        bbox_params=A.BboxParams(
            format="pascal_voc", min_area=0, min_visibility=0, label_fields=["labels"]
        ),
    )

In [32]:
from pytorch_lightning import LightningDataModule
from torch.utils.data import DataLoader

class EfficientDetDataModule(LightningDataModule):
    
    def __init__(self,
                train_dataset_adaptor,
                validation_dataset_adaptor,
                train_transforms=get_train_transform(),
                valid_transforms=get_valid_transform(),
                num_workers=4,
                height=100,
                width=100,
                batch_size=8):
        
        self.train_ds = train_dataset_adaptor
        self.valid_ds = validation_dataset_adaptor
        self.train_tfms = train_transforms
        self.valid_tfms = valid_transforms
        self.num_workers = num_workers
        self.batch_size = batch_size
        self.height = height
        self.width = width
        super().__init__()

    def train_dataset(self) -> xview:
        return xview(dataset=self.train_ds, width=self.width,
                      height=self.height,  transforms=self.train_tfms   
        )

    def train_dataloader(self) -> DataLoader:
        train_dataset = self.train_dataset()
        train_loader = torch.utils.data.DataLoader(
            train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            pin_memory=True,
            drop_last=True,
            num_workers=self.num_workers,
            collate_fn=self.collate_fn,
        )

        return train_loader

    def val_dataset(self) -> xview:
        return xview(dataset=self.valid_ds, width=self.width,
                      height=self.height,  transforms=self.valid_tfms   
        )

    def val_dataloader(self) -> DataLoader:
        valid_dataset = self.val_dataset()
        valid_loader = torch.utils.data.DataLoader(
            valid_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            pin_memory=True,
            drop_last=True,
            num_workers=self.num_workers,
            collate_fn=self.collate_fn,
        )

        return valid_loader
    
    @staticmethod
    def collate_fn(batch):
        
        images, targets, image_ids = tuple(zip(*batch))
        images = torch.stack(images)
        images = images.float()

        boxes = [target["bboxes"].float() for target in targets]
        labels = [target["labels"].float() for target in targets]
        img_size = torch.tensor([target["img_size"] for target in targets]).float()
        img_scale = torch.tensor([target["img_scale"] for target in targets]).float()

        annotations = {
            "bbox": boxes,
            "cls": labels,
            "img_size": img_size,
            "img_scale": img_scale,
        }
        #return tuple(zip(*batch))

        return images, annotations, targets, image_ids

## Load Model

In [33]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

In [34]:
def create_model(num_classes):
    num_classes = num_classes   # 63 classes
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

## Training Model

In [35]:
import torch
from pytorch_lightning import LightningModule
#from pytorch_lightning.core.decorators import auto_move_data

class EfficientDetModel(LightningModule):
    def __init__(
        self,
        num_classes=63,
        img_size=512,
        prediction_confidence_threshold=0.2,
        learning_rate=0.0002,
        wbf_iou_threshold=0.44,
        inference_transforms=get_valid_transform, #get_valid_transforms(target_img_size=512),
        model_architecture='tf_efficientnetv2_l',
    ):
        super().__init__()
        self.img_size = img_size
        self.model = create_model(num_classes)
        self.prediction_confidence_threshold = prediction_confidence_threshold
        self.lr = learning_rate
        self.wbf_iou_threshold = wbf_iou_threshold
        self.inference_tfms = inference_transforms


    #@auto_move_data
    def forward(self, images, targets):
        return self.model(images, targets)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.model.parameters(), lr=self.lr)


    def training_step(self, batch, batch_idx):
        images, annotations, _, image_ids = batch

        losses = self.model(images, annotations)

        '''      logging_losses = {
            "class_loss": losses["class_loss"].detach(),
            "box_loss": losses["box_loss"].detach(),
        }'''

        self.log("train_loss", losses["loss"], on_step=True, on_epoch=True, prog_bar=True,
                 logger=True)
        self.log(
            "train_class_loss", losses["class_loss"], on_step=True, on_epoch=True, prog_bar=True,
            logger=True
        )
        self.log("train_box_loss", losses["box_loss"], on_step=True, on_epoch=True, prog_bar=True,
                 logger=True)

        return losses['loss']


    @torch.no_grad()
    def validation_step(self, batch, batch_idx):
        images, annotations, targets, image_ids = batch
        outputs = self.model(images, annotations)

        detections = outputs["detections"]

        batch_predictions = {
            "predictions": detections,
            "targets": targets,
            "image_ids": image_ids,
        }

        logging_losses = {
            "class_loss": outputs["class_loss"].detach(),
            "box_loss": outputs["box_loss"].detach(),
        }

        self.log("valid_loss", outputs["loss"], on_step=True, on_epoch=True, prog_bar=True,
                 logger=True, sync_dist=True)
        self.log(
            "valid_class_loss", logging_losses["class_loss"], on_step=True, on_epoch=True,
            prog_bar=True, logger=True, sync_dist=True
        )
        self.log("valid_box_loss", logging_losses["box_loss"], on_step=True, on_epoch=True,
                 prog_bar=True, logger=True, sync_dist=True)

        return {'loss': outputs["loss"], 'batch_predictions': batch_predictions}


In [36]:
from pytorch_lightning import Trainer
trainer = Trainer(max_epochs=5,
    accelerator="auto",
    devices="auto",
    strategy="auto",
    )
   

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [37]:
model_eff = EfficientDetModel()

In [38]:
dataload = EfficientDetDataModule(train_dataset, test_dataset)

In [39]:
dataload

In [40]:
trainer.fit(model_eff,train_dataloaders=dataload.train_dataloader(), 
                val_dataloaders=dataload.val_dataloader())

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | FasterRCNN | 41.6 M
-------------------------------------
41.4 M    Trainable params
222 K     Non-trainable params
41.6 M    Total params
166.447   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

AttributeError: 'str' object has no attribute 'items'

In [11]:
train_dataset_1 = xview(train_dataset, 50, 50,  transforms=get_train_transform())
test_dataset_1 = xview(test_dataset, 50, 50,  transforms=get_valid_transform())
train_loader = create_train_loader(train_dataset_1)
valid_loader = create_valid_loader(test_dataset_1)
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(test_dataset)}\n")
# initialize the model and move to the computation device
model = create_model(63)
model = model.to(DEVICE)

Number of training samples: 211
Number of validation samples: 212



In [12]:
from tqdm.auto import tqdm

# function for running training iterations
def train(train_data_loader, model):
    print('Training')
    global train_itr
    global train_loss_list
    
     # initialize tqdm progress bar
    prog_bar = tqdm(train_data_loader, total=len(train_data_loader))
    
    for i, data in enumerate(prog_bar):
        optimizer.zero_grad()
        
        images, targets = data
        
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        #train_loss_list.append(loss_value)
        #train_loss_hist.send(loss_value)
        losses.backward()
        optimizer.step()
        train_itr += 1
    
        # update the loss value beside the progress bar for each iteration
        prog_bar.set_description(desc=f"Loss: {loss_value:.4f}")
    return train_loss_list

In [13]:
def validate(valid_data_loader, model):
    print('Validating')
    global val_itr
    global val_loss_list
    
    # initialize tqdm progress bar
    prog_bar = tqdm(valid_data_loader, total=len(valid_data_loader))
    
    for i, data in enumerate(prog_bar):
        images, targets = data
        
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
        
        with torch.no_grad():
            loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        val_loss_list.append(loss_value)
        val_loss_hist.send(loss_value)
        val_itr += 1
        # update the loss value beside the progress bar for each iteration
        prog_bar.set_description(desc=f"Loss: {loss_value:.4f}")
    return val_loss_list

In [14]:
import time
NUM_EPOCHS= 1
# get the model parameters
params = [p for p in model.parameters() if p.requires_grad]
# define the optimizer
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
# initialize the Averager class
#train_loss_hist = Averager()
#val_loss_hist = Averager()
train_itr = 1
val_itr = 1
# train and validation loss lists to store loss values of all...
# ... iterations till ena and plot graphs for all iterations
train_loss_list = []
val_loss_list = []
# name to save the trained model with
MODEL_NAME = 'model'

# initialize SaveBestModel class
#save_best_model = SaveBestModel()
# start the training epochs
for epoch in range(NUM_EPOCHS):
    print(f"\nEPOCH {epoch+1} of {NUM_EPOCHS}")
    # reset the training and validation loss histories for the current epoch
    #train_loss_hist.reset()
    #val_loss_hist.reset()
    # start timer and carry out training and validation
    start = time.time()
    train_loss = train(train_loader, model)
    val_loss = validate(valid_loader, model)
    #print(f"Epoch #{epoch+1} train loss: {train_loss_hist.value:.3f}")   
    #print(f"Epoch #{epoch+1} validation loss: {val_loss_hist.value:.3f}")   
    end = time.time()
    print(f"Took {((end - start) / 60):.3f} minutes for epoch {epoch}")
    # save the best model till now if we have the least loss in the...
    # ... current epoch
    save_best_model(
        val_loss_hist.value, epoch, model, optimizer
    )
    # save the current epoch model
    save_model(epoch, model, optimizer)
    # save loss plot
    save_loss_plot(OUT_DIR, train_loss, val_loss)
    
    # sleep for 5 seconds after each epoch
    time.sleep(5)


EPOCH 1 of 1
Training


  0%|          | 0/211 [00:00<?, ?it/s]

Validating


  0%|          | 0/212 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fa32117ac20>
Traceback (most recent call last):
  File "/home/chris/.virtualenvs/cdao/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/home/chris/.virtualenvs/cdao/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fa32117ac20>
Traceback (most recent call last):
  File "/home/chris/.virtualenvs/cdao/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/home/chris/.virtualenvs/cdao/lib/python3.10/site-packages/torch/utils/data/dataloader

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.42 GiB (GPU 0; 4.00 GiB total capacity; 27.89 GiB already allocated; 0 bytes free; 31.77 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF